# Soccer Player Performance Prediction


R-squared (R2) and Root Mean Square Error (RMSE) are both metrics used in regression analysis to evaluate the performance of a model. However, they measure different aspects of the model's performance.

R-squared is a measure of how well the model fits the data. It is a statistical measure that indicates the percentage of the variance in the dependent variable that is explained by the independent variable(s). R-squared values range from 0 to 1, with higher values indicating a better fit.

RMSE, on the other hand, is a measure of the difference between the predicted values of the model and the actual values of the data. It is calculated by taking the square root of the average of the squared differences between the predicted values and the actual values. RMSE values range from 0 to infinity, with lower values indicating a better fit.

In summary, R-squared measures the goodness of fit of the model, while RMSE measures the accuracy of the model's predictions. Therefore, it is possible to have a model with a high R-squared value but a high RMSE, indicating that the model is a good fit but the predictions are not accurate. Conversely, it is also possible to have a model with a low R-squared value but a low RMSE, indicating that the model is not a good fit but the predictions are accurate.





In [3]:
#importing library
import numpy as np
import pandas as pd
import sqlite3
pd.set_option('max_columns',None)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.ensemble import RandomForestRegressor

# Connecting to the SQL Server

In [8]:
connection=sqlite3.connect('/kaggle/input/soccer/database.sqlite')
players_df=pd.read_sql_query('SELECT * from Player',connection)
stats_df=pd.read_sql_query('SELECT * from Player_Attributes',connection)

In [12]:
players_df

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
...,...,...,...,...,...,...,...
11055,11071,26357,Zoumana Camara,2488,1979-04-03 00:00:00,182.88,168
11056,11072,111182,Zsolt Laczko,164680,1986-12-18 00:00:00,182.88,176
11057,11073,36491,Zsolt Low,111191,1979-04-29 00:00:00,180.34,154
11058,11074,35506,Zurab Khizanishvili,47058,1981-10-06 00:00:00,185.42,172


In [13]:
stats_df

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,183974,102359,39902,2009-08-30 00:00:00,83.0,85.0,right,medium,low,84.0,77.0,59.0,89.0,77.0,84.0,86.0,78.0,84.0,85.0,66.0,72.0,77.0,86.0,73.0,76.0,58.0,72.0,67.0,81.0,56.0,78.0,86.0,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,183975,102359,39902,2009-02-22 00:00:00,78.0,80.0,right,medium,low,74.0,76.0,53.0,84.0,77.0,85.0,86.0,74.0,73.0,86.0,66.0,67.0,77.0,74.0,73.0,75.0,58.0,66.0,65.0,73.0,61.0,64.0,72.0,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,183976,102359,39902,2008-08-30 00:00:00,77.0,80.0,right,medium,low,74.0,71.0,53.0,84.0,77.0,85.0,86.0,74.0,73.0,86.0,66.0,67.0,77.0,74.0,73.0,75.0,58.0,66.0,65.0,73.0,67.0,64.0,72.0,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,183977,102359,39902,2007-08-30 00:00:00,78.0,81.0,right,medium,low,74.0,64.0,57.0,86.0,77.0,87.0,86.0,73.0,73.0,91.0,61.0,60.0,77.0,69.0,73.0,72.0,58.0,67.0,59.0,78.0,63.0,63.0,68.0,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


# Preprocess

In [124]:
def process_inputs(players_df,stats_df):
    players_df=players_df.copy()
    stats_df=stats_df.copy()
    #droppping the unnecessary column
    players_df=players_df.drop(['id','player_name','player_fifa_api_id'],axis=1)
    stats_df=stats_df.drop(['id','player_fifa_api_id','date'],axis=1)
    #extracinting the month,year,date from the birthday column
    players_df['BirthMonth']=pd.to_datetime(players_df['birthday']).dt.month.astype(np.int)
    players_df['BirthYear']=pd.to_datetime(players_df['birthday']).dt.year.astype(np.int)
    players_df['BirthDay']=pd.to_datetime(players_df['birthday']).dt.day.astype(np.int)
    players_df=players_df.drop('birthday',axis=1)
    stats_df['preferred_foot']=stats_df['preferred_foot'].fillna(stats_df['preferred_foot'].mode()[0])
    stats_df['defensive_work_rate']=stats_df['defensive_work_rate'].apply(lambda x:np.NaN if x not in ['medium','low','high'] else x)
    stats_df['defensive_work_rate']=stats_df['defensive_work_rate'].fillna(stats_df['defensive_work_rate'].mode()[0])
    stats_df['attacking_work_rate']=stats_df['attacking_work_rate'].apply(lambda x:np.NaN if x not in ['medium','low','high'] else x)
    stats_df['attacking_work_rate']=stats_df['attacking_work_rate'].fillna(stats_df['attacking_work_rate'].mode()[0])
    stats_df['preferred_foot']=stats_df['preferred_foot'].apply(lambda x:0 if x=='left' else 1)
    intensity={'low':1,'medium':2,'high':3}
    stats_df['defensive_work_rate']=stats_df['defensive_work_rate'].replace(intensity)
    stats_df['attacking_work_rate']=stats_df['attacking_work_rate'].replace(intensity)
    df=pd.merge(stats_df,players_df,on='player_api_id',how='left')
    for column in df.columns:
        df[column]=df[column].fillna(df[column].mean())
    
    y=df['overall_rating']
    x=df.drop(['overall_rating','player_api_id'],axis=1)
    x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7)
    scaler=StandardScaler()
    scaler.fit(x_train)
    x_train=pd.DataFrame(scaler.transform(x_train),columns=x.columns)
    x_test=pd.DataFrame(scaler.transform(x_test),columns=x.columns)
    
    
    
    
    
    
    return x_train,x_test,y_train,y_test
    

In [125]:
x_train,x_test,y_train,y_test=process_inputs(players_df,stats_df)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Depreca

In [126]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(128784, 42)
(55194, 42)
(128784,)
(55194,)


In [127]:
x_train

,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,height,weight,BirthMonth,BirthYear,BirthDay
0,1.296319,0.568109,-0.375812,-0.095189,0.864857,1.159815,0.651296,0.604963,1.077330,0.891839,0.497507,0.259851,0.064316,1.160745,1.104004,1.429056,0.778631,1.522744,0.294042,1.004934,0.550932,0.454934,0.961231,0.580769,0.065517,-0.668213,1.096660,0.940099,1.029806,-0.789125,-0.992828,-1.254422,-0.517478,-0.194111,-0.699968,-0.134046,-0.608101,0.155491,-0.517505,-0.571586,0.862890,-0.505603
1,-0.377003,0.568109,1.644923,-2.115430,-1.340968,0.739367,0.287745,-0.595410,0.691456,0.215471,-0.163931,-0.525322,-1.953730,-0.091491,0.487391,0.552669,-0.153940,-0.232694,-0.245918,0.570822,1.193297,0.226369,0.545727,0.199377,-0.806071,-1.388962,0.337683,-0.921527,0.386315,-1.024907,-1.365442,-1.068220,-0.221328,-0.572286,-0.373249,-0.196158,-0.608101,0.155491,-0.517505,0.884907,0.862890,0.171908
2,-0.833363,-1.760227,-0.375812,-0.095189,0.458520,-0.732199,-0.742315,-0.030529,-0.576417,-0.460896,-0.163931,-0.805740,-0.353211,-0.289212,0.024931,-0.164376,0.079203,-0.232694,0.525453,-0.359420,-0.183200,-1.068837,-0.285283,-0.835828,-0.494790,0.001053,-0.963421,-0.921527,-0.965016,0.342629,0.590783,0.142095,-0.043639,-0.320170,-0.513271,-0.568834,-0.375816,-0.638149,-0.252426,0.884907,1.273707,1.639849
3,0.535718,-1.760227,-0.375812,1.925052,-0.121960,0.371476,0.711887,0.746183,-0.190543,0.271835,0.277028,-0.132735,0.133904,0.303952,0.256161,0.074639,-0.620225,-0.013264,0.756864,0.198725,0.092099,0.683500,0.628827,-0.073045,0.314542,1.133658,0.934022,0.940099,0.772409,1.285758,1.056551,1.073106,-0.221328,0.310121,1.773764,0.300743,0.262968,1.345951,0.940429,1.467504,-0.780378,0.736501
4,-1.441844,0.568109,-0.375812,-0.095189,-0.296104,0.424032,0.105970,-0.101139,0.029957,-0.179077,0.938465,1.157191,0.481843,-0.355119,-0.206298,-0.084704,-0.231654,-0.013264,-0.400192,0.198725,-0.091434,0.150180,0.047121,0.689738,-0.681559,0.052535,-0.096019,0.408206,0.515013,-0.694812,0.171592,0.002443,-0.399018,-0.194111,-0.326574,-0.693059,-0.143531,0.552311,1.404317,1.467504,0.246664,0.623582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128779,0.992079,-1.760227,-0.375812,-0.095189,0.748760,-0.784755,0.045378,0.534353,0.581206,0.384563,0.773106,0.540270,0.899370,0.435766,-0.206298,-0.164376,-0.153940,0.206166,-0.168781,0.198725,0.459165,0.835877,0.795029,0.689738,-0.806071,0.412910,0.229257,0.142259,0.450664,-0.317561,0.544207,0.654151,-0.102868,-0.131082,-0.419923,-0.444608,-0.433888,0.155491,0.476541,-1.445482,1.273707,0.510664
128780,0.687838,-1.760227,-0.375812,-0.095189,0.922905,0.739367,1.257213,1.240454,0.636331,0.215471,0.773106,0.708521,1.386484,0.831209,-0.437528,-1.200106,-0.153940,1.303314,-0.323055,0.384773,0.367399,0.759688,0.795029,0.635253,0.563568,1.596997,-0.638145,1.206046,0.901107,1.238601,1.429166,1.352410,0.015591,-0.635315,-0.466597,-0.134046,-0.550030,1.345951,0.940429,-1.445482,-0.985787,0.623582
128781,-2.506685,0.568109,-0.375812,-0.095189,-1.631208,-1.783318,-2.196517,-2.148834,-1.734040,-2.095451,-0.274170,-0.132735,-2.162494,-1.409633,-0.129222,-0.164376,-0.309368,-1.000698,-0.400192,-3.088128,0.550932,-0.383140,-0.866989,-1.816549,0.314542,0.670320,-1.830824,-0.855040,-0.900667,0.719881,0.870244,0.933455,-0.221328,-0.257141,-0.373249,-0.382496,-0.201602,-0.638149,-0.384965,-1.154183,0.041256,-1.521869
128782,1.600559,0.568109,1.644923,1.925052,0.806808,1.685374,1.317805,0.675573,0.967080,0.948203,0.828226,1.325442,0.412255,0.897

# Training the Model

In [128]:
model=RandomForestRegressor()
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.9806914298676771

In [135]:
r2_score=model.score(x_test,y_test)

In [129]:
y_test

140350    86.0
138710    67.0
58552     64.0
116176    53.0
145039    73.0
          ... 
25480     87.0
173180    67.0
27804     64.0
90369     73.0
173668    62.0
Name: overall_rating, Length: 55194, dtype: float64

In [132]:
y_pred=model.predict(x_test)

In [134]:
root_mean_square_error=np.sqrt(np.mean((y_pred-y_test)**2))

In [136]:
print("R2 Score",r2_score)
print('Root Mean Square Error',root_mean_square_error)

R2 Score 0.9806914298676771
Root Mean Square Error 0.9785126773471359


In [88]:
y.isna().sum()

player_api_id             0
overall_rating          836
potential               836
preferred_foot            0
attacking_work_rate       0
defensive_work_rate       0
crossing                836
finishing               836
heading_accuracy        836
short_passing           836
volleys                2713
dribbling               836
curve                  2713
free_kick_accuracy      836
long_passing            836
ball_control            836
acceleration            836
sprint_speed            836
agility                2713
reactions               836
balance                2713
shot_power              836
jumping                2713
stamina                 836
strength                836
long_shots              836
aggression              836
interceptions           836
positioning             836
vision                 2713
penalties               836
marking                 836
standing_tackle         836
sliding_tackle         2713
gk_diving               836
gk_handling         

In [78]:
{column:list(y[column].unique()) for column in y.columns if y[column].dtypes=='object'}

{}

In [50]:
{column:y[column].value_counts() for column in y.columns if y[column].dtypes=='object'}

{'preferred_foot': right    139245
 left      44733
 Name: preferred_foot, dtype: int64,
 'attacking_work_rate': medium    125070
 high       42823
 low         8569
 None        3639
 norm         348
 y            106
 le           104
 stoc          89
 Name: attacking_work_rate, dtype: int64,
 'defensive_work_rate': medium    130846
 high       27041
 low        18432
 _0          2394
 o           1550
 1            441
 ormal        348
 2            342
 3            258
 5            234
 7            217
 0            197
 6            197
 9            152
 4            116
 es           106
 ean          104
 tocky         89
 8             78
 Name: defensive_work_rate, dtype: int64}

In [48]:
y['attacking_work_rate'].value_counts()

medium    125070
high       42823
low         8569
None        3639
norm         348
y            106
le           104
stoc          89
Name: attacking_work_rate, dtype: int64

In [68]:
y['defensive_work_rate'].apply(lambda x:np.NaN if x not in ['medium','low','high'] else x).isna().sum()

7659

In [79]:
y

,player_api_id,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,67.0,71.0,1,2,2,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,505942,67.0,71.0,1,2,2,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,505942,62.0,66.0,1,2,2,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,505942,61.0,65.0,1,2,2,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,505942,61.0,65.0,1,2,2,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,39902,83.0,85.0,1,2,1,84.0,77.0,59.0,89.0,77.0,84.0,86.0,78.0,84.0,85.0,66.0,72.0,77.0,86.0,73.0,76.0,58.0,72.0,67.0,81.0,56.0,78.0,86.0,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,39902,78.0,80.0,1,2,1,74.0,76.0,53.0,84.0,77.0,85.0,86.0,74.0,73.0,86.0,66.0,67.0,77.0,74.0,73.0,75.0,58.0,66.0,65.0,73.0,61.0,64.0,72.0,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,39902,77.0,80.0,1,2,1,74.0,71.0,53.0,84.0,77.0,85.0,86.0,74.0,73.0,86.0,66.0,67.0,77.0,74.0,73.0,75.0,58.0,66.0,65.0,73.0,67.0,64.0,72.0,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,39902,78.0,81.0,1,2,1,74.0,64.0,57.0,86.0,77.0,87.0,86.0,73.0,73.0,91.0,61.0,60.0,77.0,69.0,73.0,72.0,58.0,67.0,59.0,78.0,63.0,63.0,68.0,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [ ]:
if df['medium']